In [5]:
import PySAM.Pvwattsv8 as pv
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv

# === Parámetros por sitio ===
site_params = {
    "calama_clean.csv": {
        "tilt": 22.5, "azimuth": 0,
        "lat": -22.466, "lon": -68.933, "tz": -4, "elev": 2260
    },
    "salvador_clean.csv": {
        "tilt": 26.2, "azimuth": 0,
        "lat": -26.029, "lon": -69.622, "tz": -4, "elev": 1600
    },
    "Vallenar_clean.csv": {
        "tilt": 28.6, "azimuth": 0,
        "lat": -28.576, "lon": -70.760, "tz": -4, "elev": 520
    },
}

# === Almacén de resultados agregados ===
resultados_totales = {}

def run_pv_simulation(csv_path, tilt, azimuth, lat, lon, tz, elev,
                      ac_capacity_mw=50, ac_dc_ratio=1.2, losses=14,
                      module_type=0, array_type=1):

    print(f"\n🔍 Procesando {csv_path} ...")

    # === Cargar y limpiar ===
    df = pd.read_csv(csv_path)
    columnas_criticas = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'GHI']
    df = df.dropna(subset=columnas_criticas)

    print(f"🔎 Registros tras limpieza: {len(df)}")
    if df.empty:
        raise ValueError(f"❌ El archivo '{csv_path}' no contiene datos suficientes tras limpieza.")

    # === Crear recurso solar ===
    weather = {
        'year': df['Year'].tolist(),
        'month': df['Month'].tolist(),
        'day': df['Day'].tolist(),
        'hour': df['Hour'].tolist(),
        'minute': df['Minute'].tolist(),
        'dn': df['DNI'].tolist(),
        'df': df['DHI'].tolist(),
        'gh': df['GHI'].tolist(),
        'tdry': df.get('Tdry', pd.Series([25]*len(df))).fillna(25).tolist(),
        'wspd': df.get('Wspd', pd.Series([2]*len(df))).fillna(2).tolist(),
        'pres': df.get('Pres', pd.Series([1013]*len(df))).fillna(1013).tolist(),
        'tdew': df.get('Tdew', pd.Series([10]*len(df))).fillna(10).tolist(),
        'rh': df.get('RH', pd.Series([50]*len(df))).fillna(50).tolist(),
        'wdir': df.get('Wdir', pd.Series([180]*len(df))).fillna(180).tolist(),
        'tz': tz,
        'lat': lat,
        'lon': lon,
        'elev': elev,
    }

    # === Configuración del sistema ===
    dc_capacity_kw = ac_capacity_mw * ac_dc_ratio * 1000
    sim = pv.default("PVWattsNone")
    sim.SystemDesign.system_capacity = dc_capacity_kw
    sim.SystemDesign.dc_ac_ratio = ac_dc_ratio
    sim.SystemDesign.losses = losses
    sim.SystemDesign.module_type = module_type
    sim.SystemDesign.array_type = array_type
    sim.SystemDesign.tilt = tilt
    sim.SystemDesign.azimuth = azimuth

    sim.SolarResource.solar_resource_data = weather
    sim.execute()

    # === Resultados principales ===
    print(f"📈 Resultados de {csv_path}:")
    print(f"  🔹 Producción anual (AC): {sim.Outputs.ac_annual:,.2f} kWh")
    print(f"  🔹 Factor de capacidad   : {sim.Outputs.capacity_factor:.2f}%")
    print(f"  🔹 Producción mensual    : {sim.Outputs.ac_monthly}")
    print("-" * 50)

    # === Gráfico mensual ===
    meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
             'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
    ac_mensual = sim.Outputs.ac_monthly
    sitio = os.path.basename(csv_path).replace("_clean.csv", "").capitalize()

    plt.figure(figsize=(10,5))
    plt.bar(meses, ac_mensual, color='seagreen', edgecolor='black')
    plt.title(f'{sitio} - Producción mensual de energía (AC)')
    plt.ylabel('Energía (kWh)')
    plt.xlabel('Mes')
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    nombre_salida = f"{sitio.lower()}_resultados.png"
    plt.savefig(nombre_salida)
    plt.close()
    print(f"📷 Gráfico individual guardado como: {nombre_salida}")

    # === Guardar CSV con columna 'AC' horaria ===
    ac_horaria = sim.Outputs.ac
    if len(ac_horaria) == len(df):
        df['AC'] = ac_horaria
        output_file = csv_path.replace("_clean.csv", "_simulado.csv")
        df.to_csv(output_file, index=False)
        print(f"💾 CSV con columna 'AC' guardado como: {output_file}")
    else:
        print("⚠️ No se guardó el CSV con 'AC' porque el número de horas no coincide.")

    # === Guardar para uso comparativo ===
    resultados_totales[sitio] = {
        "ac_monthly": list(sim.Outputs.ac_monthly),
        "capacity_factor": sim.Outputs.capacity_factor
    }

# === Ejecutar simulaciones ===
for site, params in site_params.items():
    run_pv_simulation(
        csv_path=site,
        tilt=params["tilt"],
        azimuth=params["azimuth"],
        lat=params["lat"],
        lon=params["lon"],
        tz=params["tz"],
        elev=params["elev"]
    )

# === Gráficos comparativos ===
def generar_graficos_comparativos(resultados):
    df_mensual = pd.DataFrame({site: datos["ac_monthly"] for site, datos in resultados.items()},
                              index=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                                     'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])

    plt.figure(figsize=(10,6))
    df_mensual.plot(kind='line', marker='o')
    plt.title("Comparación de Producción Mensual de Energía (AC)")
    plt.ylabel("Energía (kWh)")
    plt.xlabel("Mes")
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend(title="Sitio")
    plt.tight_layout()
    plt.savefig("produccion_mensual_comparativa.png")
    plt.close()
    print("📊 Gráfico 'produccion_mensual_comparativa.png' generado.")

    sitios = list(resultados.keys())
    fc_values = [resultados[sitio]["capacity_factor"] for sitio in sitios]

    plt.figure(figsize=(8,5))
    plt.bar(sitios, fc_values, color=['seagreen', 'steelblue', 'orange'], edgecolor='black')
    plt.title("Comparación del Factor de Capacidad")
    plt.ylabel("Factor de Capacidad (%)")
    plt.ylim(0, max(fc_values)*1.2)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig("factor_capacidad_comparativo.png")
    plt.close()
    print("📊 Gráfico 'factor_capacidad_comparativo.png' generado.")

generar_graficos_comparativos(resultados_totales)

# === Guardar producción anual por sitio ===
with open("produccion_anual_por_sitio.csv", "w", newline="") as csvfile:
    fieldnames = ["Sitio", "Produccion_Anual_kWh", "Capacity_Factor"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for sitio, datos in resultados_totales.items():
        produccion_anual = sum(datos["ac_monthly"])
        writer.writerow({
            "Sitio": sitio,
            "Produccion_Anual_kWh": round(produccion_anual, 2),
            "Capacity_Factor": round(datos["capacity_factor"], 2)
        })
        print(f"📝 {sitio}: {round(produccion_anual, 2)} kWh — FC: {round(datos['capacity_factor'], 2)}%")

print("✅ Archivo 'produccion_anual_por_sitio.csv' generado.")



🔍 Procesando calama_clean.csv ...
🔎 Registros tras limpieza: 8760
📈 Resultados de calama_clean.csv:
  🔹 Producción anual (AC): 100,129,671.95 kWh
  🔹 Factor de capacidad   : 19.05%
  🔹 Producción mensual    : (6763830.790982245, 7533878.436878167, 9225625.015759923, 8819807.101055998, 9389799.628406784, 8536417.4231165, 8298577.34456277, 9070339.2738963, 8410586.31207229, 7696615.10332569, 8263563.593526969, 8120631.92145046)
--------------------------------------------------
📷 Gráfico individual guardado como: calama_resultados.png
💾 CSV con columna 'AC' guardado como: calama_simulado.csv

🔍 Procesando salvador_clean.csv ...
🔎 Registros tras limpieza: 8760
📈 Resultados de salvador_clean.csv:
  🔹 Producción anual (AC): 90,934,979.79 kWh
  🔹 Factor de capacidad   : 17.30%
  🔹 Producción mensual    : (7481623.3561442215, 7007179.344643104, 7627879.955304882, 7517876.394581257, 7330880.204299693, 6942327.973378814, 7445064.473329215, 7181780.122615848, 9530521.06139978, 8470747.909650123

<Figure size 1000x600 with 0 Axes>